In [1]:
import numpy as np, pandas as pd
from scipy.sparse import coo_matrix
from poismf import PoisMF
from pandas.api.types import CategoricalDtype
from scipy import sparse
from sklearn.model_selection import train_test_split
import tqdm
from recsys_metrics import *
import torch
# https://github.com/zuoxingdong/recsys_metrics?tab=readme-ov-file#Citation


In [2]:
train_df = pd.read_csv("pf_last_lastfm_train.tsv",sep="\t",    names=[
        'UserId', 'ItemId', 'Count', 'timestamp'
    ],)
test_df = pd.read_csv("pf_last_lastfm_test.tsv",sep="\t",    names=[
        'UserId', 'ItemId', 'Count', 'timestamp'
    ],)


In [3]:
display(test_df)
display(train_df)

,UserId,ItemId,Count,timestamp
0,0,106735,1,NaN
1,1,716377,8,NaN
2,2,29778,4,NaN
3,3,285398,1,NaN
4,4,466528,1,NaN
...,...,...,...,...
1187337,683,541596,2,NaN
1187338,291,385161,2,NaN
1187339,902,70159,4,NaN
1187340,547,22609,1,NaN


,UserId,ItemId,Count,timestamp
0,0,929335,27,NaN
1,0,294494,7,NaN
2,0,942636,8,NaN
3,0,64905,8,NaN
4,0,928556,7,NaN
...,...,...,...,...
2770460,991,734114,1,NaN
2770461,991,689734,1,NaN
2770462,991,896797,1,NaN
2770463,991,920272,1,NaN


In [4]:
test_users =test_df["UserId"].unique()
test_userDict ={}
for i in test_users:
    test_userDict[i] ={}

In [5]:
train_users =train_df["UserId"].unique()
train_userDict ={}
for i in train_users:
    train_userDict[i] ={}

In [6]:
for index, row in test_df.iterrows():
    user = row["UserId"]
    item = row["ItemId"]
    rating = row["Count"]
    if item in test_userDict[user]:
        test_userDict[user][item]+=1
    else:
        test_userDict[user][item]=1

In [7]:
for index, row in train_df.iterrows():
    user = row["UserId"]
    item = row["ItemId"]
    rating = row["Count"]
    if item in train_userDict[user]:
        train_userDict[user][item]+=1
    else:
        train_userDict[user][item]=1

In [8]:
train_res = []
for user, items in train_userDict.items():
    for item, rating in items.items():
        train_res.append((user, item, rating))

train_df = pd.DataFrame(train_res, columns=["UserId", "ItemId", "Count"])

In [9]:
test_res = []
for user, items in test_userDict.items():
    for item, rating in items.items():
        test_res.append((user, item, rating))

test_df = pd.DataFrame(test_res, columns=["UserId", "ItemId", "Count"])

In [10]:
display(train_df)
display(test_df)

,UserId,ItemId,Count
0,0,929335.0,1
1,0,294494.0,1
2,0,942636.0,1
3,0,64905.0,1
4,0,928556.0,1
...,...,...,...
2770460,991,734114.0,1
2770461,991,689734.0,1
2770462,991,896797.0,1
2770463,991,920272.0,1


,UserId,ItemId,Count
0,0,106735.0,1
1,0,551.0,1
2,0,3080.0,1
3,0,3300.0,1
4,0,3623.0,1
...,...,...,...
1187337,991,728668.0,1
1187338,991,700535.0,1
1187339,991,109353.0,1
1187340,991,666164.0,1


In [11]:
user_positives = test_df.groupby('UserId')['ItemId'].apply(set).to_dict()


In [12]:
test_users = test_df['UserId'].unique()
test_items = test_df['ItemId'].unique()


train_users = train_df['UserId'].unique()
train_items = train_df['ItemId'].unique()


user = list(set(train_users) & set(test_users))
item = list(set(train_items) & set(test_items))

In [13]:
train_df = train_df[train_df['UserId'].isin(user)]
train_df = train_df[train_df['ItemId'].isin(item)]

test_df = test_df[test_df['UserId'].isin(user)]
test_df = test_df[test_df['ItemId'].isin(item)]

In [14]:
train_df["Count"] =1
test_df["Count"] =1


In [15]:

test_ratings_matrix = test_df.pivot(index='UserId', columns='ItemId', values='Count')
train_ratings_matrix = train_df.pivot(index='UserId', columns='ItemId', values='Count')


In [16]:
display(train_ratings_matrix)
display(test_ratings_matrix)

ItemId,0.0,1.0,3.0,5.0,6.0,7.0,11.0,12.0,14.0,15.0,...,960378.0,960380.0,960381.0,960384.0,960386.0,960387.0,960389.0,960394.0,960398.0,960400.0
UserId,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


ItemId,0.0,1.0,3.0,5.0,6.0,7.0,11.0,12.0,14.0,15.0,...,960378.0,960380.0,960381.0,960384.0,960386.0,960387.0,960389.0,960394.0,960398.0,960400.0
UserId,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
modelPF = PoisMF(reindex=True, method="tncg", use_float=True,
                    early_stop=False, reuse_prev=True,
                    k=20, niter=10, maxupd=200, l2_reg=1e3)\
                .fit(train_df)

In [18]:
del train_df
del test_df
del train_ratings_matrix
del user
del item
del test_res
del train_res
del test_userDict
del train_userDict
del test_users
del train_users


In [19]:
display(modelPF.A.shape) # user
display(modelPF.B.shape) # items


(991, 20)

(445479, 20)

In [20]:
pred = np.matmul(modelPF.A,modelPF.B.T)

In [21]:
display(pred.shape)

(991, 445479)

In [22]:
test_ratings_matrix[test_ratings_matrix >1] = 1

true = torch.tensor(test_ratings_matrix.fillna(0).to_numpy())


In [23]:
predv = torch.tensor(pred)


In [24]:
del modelPF
del pred
del test_ratings_matrix

In [25]:
display(true)
display(predv)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

tensor([[8.0394e-03, 2.0507e-02, 1.7907e-02,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.8517e-04, 0.0000e+00, 5.0093e-10,  ..., 4.4074e-03, 4.4074e-03,
         4.4066e-03],
        [1.8217e-04, 4.6470e-04, 4.0577e-04,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [4.0035e-03, 3.2636e-03, 2.8497e-03,  ..., 7.6392e-03, 7.6392e-03,
         7.6378e-03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.9308e-03, 5.2530e-03, 4.5869e-03,  ..., 6.2284e-03, 6.2284e-03,
         6.2273e-03]])

In [26]:
def calculate_metrics(pred_df, user_postivies, k_list = [10,50,100]):
  metrics = {
    'MAR@ALL': [],
    'MAR@1000': [],
  }


  grouped = pred_df.groupby('user')

  for user, user_data in tqdm.tqdm(grouped, desc="Processing Users"):
    positives = user_postivies.get(user, set())
    if len(positives) == 0:
      continue

    sorted_data = user_data.sort_values(by='score', ascending=False)
    all_items = sorted_data['item'].tolist()

    relevant_ranks1k = []
    relevant_ranks_all=[]
    for rank, item in enumerate(all_items[:1000], start=1):
      if item in positives:
        relevant_ranks1k.append(rank)
    #calculate relevant
    for rank, item in enumerate(all_items, start=1):
      if item in positives:
        relevant_ranks_all.append(rank)



    if relevant_ranks_all:
      metrics['MAR@ALL'].append(np.mean(relevant_ranks_all))
    if relevant_ranks1k:
      metrics['MAR@1000'].append(np.mean(relevant_ranks1k))
      

  
  final = {
    'MAR@ALL': np.mean(metrics['MAR@ALL']),
    'MAR@1000': np.mean(metrics['MAR@1000']),
  }

  return final

In [27]:
def metricStuff(pred,true,user_positive):

    print("NDCG@10",normalized_dcg(pred, true ,k=10))
    print("MRR@1000",mean_reciprocal_rank(pred, true,k=1000))
    print("MRR@all",mean_reciprocal_rank(pred, true))
    marData = pred.detach().numpy()
    marDF = pd.DataFrame(marData)
    marDF.index.name = "user"
    marDF.columns.name = "items"
    marDF = marDF.stack().reset_index()
    marDF.columns = ['user', 'item', 'score']
    print("MAR",calculate_metrics(marDF,user_positive))
    print("MAP",mean_average_precision(pred, true))
    print("P@50",precision(pred, true,k=50))
    print("R@50",recall(pred, true,k=50))
    

In [28]:
metricStuff(predv,true,user_positives)

NDCG@10 tensor(0.0012)
MRR@1000 tensor(0.0083)
MRR@all tensor(0.0085)


Processing Users: 100%|██████████| 991/991 [02:05<00:00,  7.87it/s]


MAR {'MAR@ALL': np.float64(223843.0192641035), 'MAR@1000': np.float64(501.27488504259213)}
MAP tensor(0.0015)
P@50 tensor(0.0015)
R@50 tensor(0.0001)
